In [1]:
import urllib
from urllib.request import urlopen
import re
import requests
import time
import datetime
import pytz

In [2]:
def getHtmlFile(url,opts,verbose = 0,encoding="utf8"):
    urladdress = url + '?' + urllib.parse.urlencode(opts)
    if(verbose):
        print(urladdress)
    urlstream = urlopen(urladdress)
    htmlfile = urlstream.read().decode(encoding)
    urlstream.close()
    return htmlfile

In [3]:
def getNaverFinance(code,encoding="euc-kr"):
    url = 'https://finance.naver.com/item/main.naver'
    opts = {
         'code':code,
    }
    htmlfile = getHtmlFile(url,opts,encoding=encoding)
    return htmlfile

In [4]:
def getPrice(code):
    htmlfile=getNaverFinance(code)
    #df_list = pd.read_html(htmlfile)
    #df = df_list[1][1:] 
    #df.columns = df_list[1].iloc[0]
    
    #matches = re.findall(r'<div id="middle" class="new_totalinfo">(.+?)</dl>', htmlfile, flags=re.DOTALL)
    matches2 = re.findall(r'현재가(.+?)전일', htmlfile, flags=re.DOTALL)
    return float(matches2[0].replace(',',''))

In [5]:
getPrice(138040)

38950.0

In [43]:
df_list[3]

주요재무정보   최근 연간 실적                                   최근 분기 실적  \
       주요재무정보    2019.12    2020.12    2021.12 2022.12(E)   2021.09   
       주요재무정보     IFRS연결     IFRS연결     IFRS연결     IFRS연결    IFRS연결   
0         매출액  223151.00  284706.00  356499.00        NaN  89629.00   
1        영업이익    9623.00   13721.00   18006.00   21677.00   4885.00   
2       당기순이익    7871.00    9253.00   13832.00   16340.00   3665.00   
3       영업이익률       4.31       4.82       5.05        NaN      5.45   
4        순이익률       3.53       3.25       3.88        NaN      4.09   
5   ROE(지배주주)      15.26      16.06      22.32      28.65     20.55   
6        부채비율     935.99     911.51    1091.74        NaN   1040.10   
7        당좌비율        NaN        NaN        NaN        NaN       NaN   
8         유보율    8309.75    9535.09    9717.30        NaN   9586.49   
9      EPS(원)    2841.00    3587.00    5889.00    8034.00   1667.00   
10     PER(배)       4.15       2.74       7.47       5.02      6.34   
11     BPS(원)   20399.00   25445.00   29591.00   29166.00  28802.00   
12     PBR(배)       0.58       0.39       1.49       1.38      1.17   
13   주당배당금(원)     550.00     900.00     210.00      90.00       NaN   
14   시가배당률(%)       4.66       9.17       0.48        NaN       NaN   
15    배당성향(%)      18.61      24.19       3.35        NaN       NaN   

                                                           
      2021.12    2022.03    2022.06    2022.09 2022.12(E)  
       IFRS연결     IFRS연결     IFRS연결     IFRS연결     IFRS연결  
0   108770.00  139898.00  177949.00  211251.00        NaN  
1     4113.00    6847.00    4995.00    6258.00    3577.00  
2     3505.00    5224.00    3681.00    4863.00        NaN  
3        3.78       4.89       2.81       2.96        NaN  
4        3.22       3.73       2.07       2.30        NaN  
5       22.32      28.00      30.65      32.69        NaN  
6     1091.74    1264.48    1459.32    1540.15        NaN  
7         NaN        NaN        NaN        NaN        NaN  
8     9717.30    8984.77    8131.19    7817.89        NaN  
9     1499.00    2514.00    1766.00    2495.00    1239.00  
10       7.47       5.75       3.47       2.52      32.54  
11   29591.00   28514.00   25385.00   24610.00   29166.00  
12       1.49       1.43       1.02       0.85       1.38  
13        NaN        NaN        NaN        NaN        NaN  
14        NaN        NaN        NaN        NaN        NaN  
15        NaN        NaN        NaN        NaN        NaN

In [6]:
meritzHoldings="138040"
meritzInsuarances="000060"
meritzSecurities="008560"

exch_rate_ins=1.2657378
exch_rate_sec=0.1607327

thrshld_min=0.027
thrshld_max=0.081

In [7]:
def getDiscountRates():
    price_hol=getPrice(meritzHoldings)
    price_ins=getPrice(meritzInsuarances)
    price_sec=getPrice(meritzSecurities)

    disc_rate_ins = 1 - price_ins/price_hol/exch_rate_ins
    disc_rate_sec = 1 - price_sec/price_hol/exch_rate_sec
    
    return [disc_rate_ins,disc_rate_sec]

In [8]:
def send_email_alert(subject,text):
    #You need to use your email sender API here
    #return requests.post(
    #    "",
    #    auth=("api", ""),
    #    data={"from": "",
    #        "to": "",
    #        "subject": subject,
    #        "text": text})
    return

In [9]:
while(True):
    current_time = datetime.datetime.now(pytz.timezone('Asia/Seoul'))
    if current_time.weekday()<5 and current_time.hour>=9 and current_time.hour<16:

        #Fetch data and calculate discount rates
        [disc_rate_ins,disc_rate_sec]=getDiscountRates()
        print(current_time,'\t',[disc_rate_ins,disc_rate_sec])
            
        #See if the drs are out of the thresholds
        title=""

        if disc_rate_ins<thrshld_min:
            title+="INS EXPENSIVE "
        elif disc_rate_ins>thrshld_max:
            title+="INS CHEAP "

        if disc_rate_sec<thrshld_min:
            title+="SEC EXPENSIVE "
        elif disc_rate_sec>thrshld_max:
            title+="SEC CHEAP "

        #If true, send an email alert and sleep an hour. If not check it again after a min.
        if len(title)>0:
            send_email_alert("[Meritz]"+title,"This is an arbitrate monitoring message.\n disc_rate_ins="+str(100*disc_rate_ins)+"%\n disc_rate_sec="+str(100*disc_rate_sec)+"%")
            print("Alert sent out! Sleeping for an hour")
            time.sleep(3600)
        else :
            time.sleep(60)
    else:
        #Run it until this year
        if current_time.year>2022:
            print(current_time,"\tHappy New Year!")
            break
        
        #If the market hour is done wait until the next morning
        print(current_time,"\tWaiting for the next day.")
        time.sleep((24-16+9)*3600)

2022-12-06 03:52:36.307398+09:00 	Waiting for the next day.


KeyboardInterrupt: 